In [84]:
from osgeo import gdal
import rasterio as rio
from rasterio import features
import matplotlib.pyplot as plt
import os
import fiona
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import glob

In [85]:
catchment_dir = './carb_huc_dems/catchments'
catchment_tifs = glob.glob(os.path.join(catchment_dir, '*catchments.tif'))
catchment_tifs    

['./carb_huc_dems/catchments/140300050103-Combined-catchments.tif',
 './carb_huc_dems/catchments/060200010905-Combined-catchments.tif',
 './carb_huc_dems/catchments/140100030402-Combined-catchments.tif',
 './carb_huc_dems/catchments/030601090201-Combined-catchments.tif',
 './carb_huc_dems/catchments/110100020402-Combined-catchments.tif',
 './carb_huc_dems/catchments/060200010609-Combined-catchments.tif',
 './carb_huc_dems/catchments/060400060502-Combined-catchments.tif',
 './carb_huc_dems/catchments/020502040301-Combined-catchments.tif',
 './carb_huc_dems/catchments/110100050501-Combined-catchments.tif',
 './carb_huc_dems/catchments/060200021403-Combined-catchments.tif',
 './carb_huc_dems/catchments/030402040808-Combined-catchments.tif',
 './carb_huc_dems/catchments/103001021305-Combined-catchments.tif',
 './carb_huc_dems/catchments/130600050504-Combined-catchments.tif',
 './carb_huc_dems/catchments/130402080404-Combined-catchments.tif',
 './carb_huc_dems/catchments/031502020405-Combin

In [74]:
src = rio.open('carb_huc_dems/catchments/070600020203-Combined-catchments.tif')
x = src.read(1)
mask = x > 0


In [56]:
src.crs.to_wkt()

'PROJCS["NAD83 / Conus Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","5070"]]'

Exception ignored in: <function Collection.__del__ at 0x7f33a3a2f240>
Traceback (most recent call last):
  File "/home/mcoving/anaconda3/envs/geo_env/lib/python3.11/site-packages/fiona/collection.py", line 484, in __del__
    self.close()
  File "/home/mcoving/anaconda3/envs/geo_env/lib/python3.11/site-packages/fiona/collection.py", line 454, in close
    if self.session is not None and self.session.isactive():
       ^^^^^^^^^^^^
AttributeError: 'Collection' object has no attribute 'session'


AttributeError: 'DatasetReader' object has no attribute 'schema'

In [76]:
results = (
    {'properties': {'raster_val':v}, 'geometry': s} for i, (s,v) in enumerate(features.shapes(x, mask=mask, transform=src.transform)))

In [79]:
with fiona.open('test_catchments.shp',
                'w',
                driver='ESRI Shapefile',
                crs=src.crs.to_wkt(),
                schema={'properties':[('raster_val', 'int')],
                        'geometry': 'Polygon'}) as dst:
    dst.writerecords(results)

In [82]:
df = gpd.read_file('test_catchments.shp')
df

,raster_val,geometry
0,3,"POLYGON ((313202.957 2289557.106, 313211.248 2..."
1,2,"POLYGON ((316038.768 2289150.806, 316055.352 2..."
2,1,"POLYGON ((318584.365 2280585.329, 318609.240 2..."


In [72]:
shapes = features.shapes(x, mask=mask, transform=src.transform)
#pprint.pprint(next(shapes))
geoms = list(shapes)
geoms[0]

({'type': 'Polygon',
  'coordinates': [[(313202.9565074262, 2289557.1064008055),
    (313211.24835335417, 2289557.1064008055),
    (313211.24835335417, 2289548.8145548776),
    (313236.12389113806, 2289548.8145548776),
    (313236.12389113806, 2289532.2308630217),
    (313227.8320452101, 2289532.2308630217),
    (313227.8320452101, 2289523.9390170937),
    (313219.5401992821, 2289523.9390170937),
    (313219.5401992821, 2289515.647171166),
    (313211.24835335417, 2289515.647171166),
    (313211.24835335417, 2289474.187941526),
    (313202.9565074262, 2289474.187941526),
    (313202.9565074262, 2289424.4368659584),
    (313194.6646614983, 2289424.4368659584),
    (313194.6646614983, 2289416.1450200304),
    (313186.3728155703, 2289416.1450200304),
    (313186.3728155703, 2289382.9776363187),
    (313178.0809696423, 2289382.9776363187),
    (313178.0809696423, 2289374.6857903907),
    (313169.7891237144, 2289374.6857903907),
    (313169.7891237144, 2289366.3939444623),
    (313161.49727

In [73]:
gpd.GeoDataFrame.from_features(geoms)

TypeError: tuple indices must be integers or slices, not str

In [69]:
import pprint
shapes = features.shapes(x, mask=mask, transform=src.transform)
#pprint.pprint(next(shapes))
"""sink_nums = []
geometries = []
for geom, num in shapes:
    sink_nums.append(num)
    geometries.append(geom)
"""
#df = gpd.GeoDataFrame(sink_nums, geometry=geometries)

schema = {'geometry': 'Polygon',}
          #'properties': {'value':'int:10'}}

with fiona.open('test_catchments.shp', "w", 
                driver='ESRI Shapefile', 
                crs=src.crs.to_wkt(),
                schema=schema) as dst:
    dst.writerecords(shapes)
    #for feat in shapes:
    #    dst.write(feat)


TypeError: not all arguments converted during string formatting

In [49]:
geometries[0]

{'type': 'Polygon',
 'coordinates': [[(313202.9565074262, 2289557.1064008055),
   (313211.24835335417, 2289557.1064008055),
   (313211.24835335417, 2289548.8145548776),
   (313236.12389113806, 2289548.8145548776),
   (313236.12389113806, 2289532.2308630217),
   (313227.8320452101, 2289532.2308630217),
   (313227.8320452101, 2289523.9390170937),
   (313219.5401992821, 2289523.9390170937),
   (313219.5401992821, 2289515.647171166),
   (313211.24835335417, 2289515.647171166),
   (313211.24835335417, 2289474.187941526),
   (313202.9565074262, 2289474.187941526),
   (313202.9565074262, 2289424.4368659584),
   (313194.6646614983, 2289424.4368659584),
   (313194.6646614983, 2289416.1450200304),
   (313186.3728155703, 2289416.1450200304),
   (313186.3728155703, 2289382.9776363187),
   (313178.0809696423, 2289382.9776363187),
   (313178.0809696423, 2289374.6857903907),
   (313169.7891237144, 2289374.6857903907),
   (313169.7891237144, 2289366.3939444623),
   (313161.49727778643, 2289366.3939444

In [37]:
df[0][0].keys()

dict_keys(['type', 'coordinates'])

In [25]:
mask = x > 0
feat = features.dataset_features(src, mask=mask)
#pprint.pprint(next(feat))

TypeError: dataset_features() got an unexpected keyword argument 'mask'

In [1]:
from polygonize_catchments import polygonize_catchment_tif
tif_file = 'carb_huc_dems/catchments/070600020203-Combined-catchments.tif'
polygonize_catchment_tif(tif_file)

Polygonizing carb_huc_dems/catchments/070600020203-Combined-catchments.tif


In [ ]:
### NOtes on command lines found in history of rene (6/7/2024)
# ls > filelist.txt
# gdalbuildvrt -input_file_list filelist.txt merged-catchments.vrt


# This looks like it may have been a first attempt and I moved to using 
# polygonize-catchments.py to create a gpkg. 